# Model Optimization

## Optimization Attempt 1 Outlier Insensitivity
Our initial model uses the binary cross entropy loss function. We will investigate the output some alternative probalistic loss functions to see which minimizes the loss on our test set.
The functions in contention are: Hinge Loss , KL Divergance , Poisson Class , and Binary Cross Entropy

## Preprocessing

In [7]:
# Rebuilding the model up to the point where the loss function is implemented
import pandas as pd
# Reading in charity_data.csv as a dataframe
charity_df = pd.read_csv('Resources/charity_data.csv')

# Dropping the EIN and name columns
charity_df.drop(['EIN' , 'NAME'] , axis= 1, inplace= True)

# Find the unique Value counts in the Application_Type column
type_count = charity_df['APPLICATION_TYPE'].value_counts()

# We will set the cut off at 500 , so that application types with less than 500 applications will be aggregated into the "other" group
less_than_500 = type_count[charity_df['APPLICATION_TYPE'].value_counts() < 500].index

# replacing application type with other
charity_df['APPLICATION_TYPE'] = charity_df['APPLICATION_TYPE'].replace(less_than_500 , 'Other')

# Finding unique value counts in CLASSIFICATION
class_count = charity_df['CLASSIFICATION'].value_counts()

# Cut of will be set at 1000
less_than_1000 = class_count[charity_df['CLASSIFICATION'].value_counts() < 1000].index

# Replacing the listed classes with other
charity_df['CLASSIFICATION'] = charity_df['CLASSIFICATION'].replace(less_than_1000 , 'other')

# Convert Categorical values into numeric
numeric_df = pd.get_dummies(charity_df)

# Splitting inputs from output
y = numeric_df.pop('IS_SUCCESSFUL') # Pop off "Is_Succesful" our target column
X = numeric_df # the remaining features after the pop off

from sklearn.model_selection import train_test_split
# Splitting data into training and testing sets
X_train , X_test , y_train , y_test = train_test_split(X , y , random_state= 78)

from sklearn.preprocessing import StandardScaler
# scaling the data sets
scaler = StandardScaler()

# Fitting the scaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


## Training

In [8]:
import tensorflow as tf
# Creating our neural network
nn_model = tf.keras.models.Sequential()

# Creating our first Dense Layer
nn_model.add(tf.keras.layers.Dense(units=43 ,activation= 'relu' , input_dim = 43 )) # there are 43 features that will serve as inputs

# Adding our second layer
nn_model.add(tf.keras.layers.Dense(units=43 , activation= 'relu' , input_dim = 43))

# Creating the output layer
nn_model.add(tf.keras.layers.Dense(units = 1 , activation = 'sigmoid'))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 43)                1892      
_________________________________________________________________
dense_1 (Dense)              (None, 43)                1892      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 44        
Total params: 3,828
Trainable params: 3,828
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Testing loss functions
# generating random test ouputs
y_true = [0, 1, 0, 0]
y_pred = [-18.6, 0.51, 2.94, -12.8]

# Binary Cross Entropy
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
bce(y_true, y_pred).numpy()

# Hinge Loss

0.865458